In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math

In [2]:
%matplotlib inline

### BLS algorithm (rewritten from author's code)

*Original code*:

http://www.konkoly.hu/staff/kovacs/bls_code.html

eebls.f

In [3]:
def bls(t, x, qmi, qma, fmin, df, nf, nb):
    """Frist trial, BLS algorithm, only minor modification from author's code"""
    
    n = len(t); rn = len(x)
    #! use try
    if n != rn:
        print ("Different size of array, t and x")
        return 0

    rn = float(rn) # float of n

    minbin = 5
    nbmax = 2000
    if nb > nbmax:
        print ("Error: NB > NBMAX!")
        return 0

    tot = t[-1] - t[0] # total time span

    if fmin < 1.0/tot:
        print ("Error: fmin < 1/T")
        return 0

    # parameters in binning (after folding)
    kmi = int(qmi*nb) # nb is number of bin -> a single period
    if kmi < 1: 
        kmi = 1
    kma = int(qma*nb) + 1
    kkmi = rn*qmi # to check the bin size
    if kkmi < minbin: 
        kkmi = minbin

    # For the extension of arrays (edge effect: transit happen at the edge of data set)
    nb1 = nb + 1
    nbkma = nb + kma
        
    # Data centering
    t1 = t[0]
    u = t - t1
    s = np.mean(x) # ! Modified
    v = x - s

    bpow = 0.0
    p = np.zeros(nf)
    # Start period search
    for jf in range(nf):
        f0 = fmin + df*jf # iteration in frequency not period
        p0 = 1.0/f0

        # Compute folded time series with p0 period
        ibi = np.zeros(nbkma)
        y = np.zeros(nbkma)
        for i in range(n):
            ph = u[i]*f0 # instead of t mod P, he use t*f then calculate the phase (less computation)
            ph = ph - int(ph)
            j = int(nb*ph) # data to a bin 
            ibi[j] = ibi[j] + 1 # number of data in a bin
            y[j] = y[j] + v[i] # sum of light in a bin
        
        # Extend the arrays  ibi()  and  y() beyond nb by wrapping
        for j in range(nb1, nbkma):
            jnb = j - nb
            ibi[j] = ibi[jnb]
            y[j] = y[jnb]

        # Compute BLS statictics for this trial period
        power = 0.0

        for i in range(nb): # shift the test period
            s = 0.0
            k = 0
            kk = 0
            nb2 = i + kma
            # change the size of test period (from kmi to kma)
            for j in range(i, nb2): 
                k = k + 1
                kk = kk + ibi[j]
                s = s + y[j]
                if k < kmi: continue # only calculate SR for test period > kmi
                if kk < kkmi: continue # 
                rn1 = float(kk)
                powo = s*s/(rn1*(rn - rn1))
                if powo > power: # save maximum SR in a test period
                    power = powo # SR value
                    jn1 = i # 
                    jn2 = j
                    rn3 = rn1
                    s3 = s

        power = math.sqrt(power)
        p[jf] = power

        if power > bpow:
            bpow = power # Save the absolute maximum of SR
            in1 = jn1
            in2 = jn2
            qtran = rn3/rn
            # depth = -s3*rn/(rn3*(rn - rn3))
            # ! Modified
            high = -s3/(rn - rn3)
            low = s3/rn3
            depth = high - low
            bper = p0
    
    # ! add
    sde = (bpow - np.mean(p))/np.std(p) # signal detection efficiency

    return bpow, in1, in2, qtran, depth, bper, sde, p, high, low

### Generate simulated data with Jupiter-size planet transit

Use *batman-package* to generate light-curve (http://astro.uchicago.edu/~kreidberg/batman/)

Jupiter-size planet revolve the Solar-size star with period 2 days

In [4]:
import batman #install batman-package (Bad-Ass Transit Model cAlculatioN)
import astropy.units as u
from astropy.constants import G, R_sun, M_sun, R_jup, M_jup

ImportError: No module named batman

In [ ]:
params = batman.TransitParams() # object to store the transit parameters
params.t0 = 1.0 # time of inferior conjunction 
params.per = 3.62 # orbital period (days)
params.rp = R_jup/R_sun # planet radius (in units of stellar radii)

# calculate semi-major axis from orbital period value
a = (((params.per*u.day)**2 * G * (M_sun + M_jup) / (4*np.pi**2))**(1./3)).to(R_sun).value 

params.a = a # semi-major axis (in units of stellar radii)
params.inc = 90.0  # orbital inclination (in degrees)
params.ecc = 0. # eccentricity
params.w = 90. # longitude of periastron (in degrees), 90 for circular
params.u = [0.61] # limb darkening coefficients
params.limb_dark = "linear" # limb darkening model

t = np.linspace(0.0, 6.0, 1000) # times at which to calculate the light curve
m = batman.TransitModel(params, t) # initializes the model
f = m.light_curve(params)

# add gaussian error 
rs = np.random.RandomState(seed=13)

errors = 0.002*np.ones_like(f)
f += errors*rs.randn(len(t))

plt.plot(t, f, 'b.')
plt.title("Period: {0} d".format(params.per))

# fil = open('test.txt', 'w')
# for i in range(len(t)):
#     fil.write(str(t[i])+ ' ' +str(f[i]) + '\n')

# t = []
# f = []
# with open('test.txt', 'r') as ifile:
#     for line in ifile:
#         iline = line.strip().split()
#         t.append(float(iline[0]))
#         f.append(float(iline[1]))

# t = np.array(t)
# f = np.array(f)
# plt.plot(t, f, 'b.')
# plt.title("Period: {0} d".format(2))

plt.xlabel("t")
plt.ylabel("Relative Intensity")

### Calculate using BLS algorithm

In [ ]:
qmi = 0.01
qma = 0.1
fmin = 0.25 
df = 0.001 
nf = 1000
nb = 200
res = bls(t, f, qmi, qma, fmin, df, nf, nb)
print ("Best SR: ", res[0], "\nIngress: ", res[1], "\nEgress: ", res[2], "\nq: ", res[3], \
"\nDepth: ", res[4], "\nPeriod: ", res[5], "\nSDE: ", res[6])

In [ ]:
freq = np.arange(0.25, 1.3, 0.001)
plt.plot(freq, res[7], 'k.')
plt.title("SDE = {0}".format(res[6]))
plt.xlabel(r"Frequency (d$^{-1}$)")
plt.ylabel(r"$SR$")

## Lets explore, BLS: steps
#### Data centering
Subtract it with the average value

In [ ]:
t1 = t[0]
u = t - t1
s = np.mean(f)
v = f - s

plt.plot(u, v, 'b.')
plt.title("Data centering")
plt.xlabel(r"$t - t_0$")
plt.ylabel(r"$x(t) - \mu$")

#### Folded and binned
If we fold the data with *correct period* and binned to 200 bins

In [ ]:
f0 = 1.0/res[5] #  freq = 1/T
nbin = nb # number of bin
n = len(t)
ibi = np.zeros(nbin)
y = np.zeros(nbin)
phase = np.linspace(0.0, 1.0, nbin)

for i in range(n):
    ph = u[i]*f0 
    ph = ph - int(ph)
    j = int(nbin*ph) # data to a bin 
    ibi[j] = ibi[j] + 1.0 # number of data in a bin
    y[j] = y[j] + v[i] # sum of light in a bin


plt.plot(phase, y/ibi, 'r.')
plt.title("Period: {0} d  bin: {1}".format(1/f0, nbin))
plt.xlabel(r"Phase ($\phi$)")
plt.ylabel(r"Mean value of $x(\phi)$ in a bin")

#### Fitted to the BLS result
Folded with *correct trial period* and fit it with the BLS results.

In [ ]:
plt.plot(phase, y/ibi, 'r.')
plt.title("Period: {0} d  bin: {1}".format(1/f0, nbin))

fite = np.zeros(nbin) + res[8] # H
fite[res[1]:res[2]+1] = res[9] # L
    
plt.plot(phase, fite)
plt.xlabel(r"Phase ($\phi$)")
plt.ylabel(r"Mean value of $x(\phi)$ in a bin")

#### Folded but unbinned

What if the light curve is only folded not binned: more computation of SR

In [ ]:
T0 = res[5]
n = len(t)
y = np.zeros(n)
y = t % T0

plt.plot(y, v, 'r.')
plt.title("Period: {0} d".format(1/f0))
plt.xlabel(r"Phase ($\phi$)")
plt.ylabel(r"Mean value of $x(\phi)$ in a bin")

#### Folded with wrong trial period
More scattered

In [5]:
f0 = 0.4 #  freq = 1/T
nbin = nb # number of bin
n = len(t)
ibi = np.zeros(nbin)
y = np.zeros(nbin)
phase = np.linspace(0.0, 1.0, nbin)

for i in range(n):
    ph = u[i]*f0 
    ph = ph - int(ph)
    j = int(nbin*ph) # data to a bin 
    ibi[j] = ibi[j] + 1.0 # number of data in a bin
    y[j] = y[j] + v[i] # sum of light in a bin


plt.plot(phase, y/ibi, 'r.')
plt.title("Period: {0} d  bin: {1}".format(1/f0, nbin))
plt.xlabel(r"Phase ($\phi$)")
plt.ylabel(r"Mean value of $x(\phi)$ in a bin")

NameError: name 'nb' is not defined

#### Folded with the *alias of trial period*

Less scattered

In [ ]:
f0 = 0.25 #  freq = 1/T
nbin = nb # number of bin
n = len(t)
ibi = np.zeros(nbin)
y = np.zeros(nbin)
phase = np.linspace(0.0, 1.0, nbin)

for i in range(n):
    ph = u[i]*f0 
    ph = ph - int(ph)
    j = int(nbin*ph) # data to a bin 
    ibi[j] = ibi[j] + 1.0 # number of data in a bin
    y[j] = y[j] + v[i] # sum of light in a bin


plt.plot(phase, y/ibi, 'r.')
plt.title("Period: {0} d  bin: {1}".format(1/f0, nbin))
plt.xlabel(r"Phase ($\phi$)")
plt.ylabel(r"Mean value of $x(\phi)$ in a bin")